In [70]:
import pandas as pd
import re 
import geopandas as gpd
import pipeline_jmidkiff as pipeline

pd.set_option('display.max_rows', 100)

In [71]:
initial_assessments = pd.read_csv(
    'data/Cook_County_Assessor_s_Residential_Property_Characteristics.zip', 
    sep='\t', compression='zip')
# https://datacatalog.cookcountyil.gov/Property-Taxation/Cook-County-Assessor-s-Residential-Property-Charac/bcnq-qi2z
pipeline.show(initial_assessments)

d:\everything\python\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Shape:
(1995108, 82)

Data Types:
PIN                                                  int64
Property Class                                       int64
Tax Year                                             int64
Neighborhood Code                                    int64
Land Square Feet                                     int64
Town Code                                            int64
Type of Residence                                  float64
Apartments                                         float64
Wall Material                                      float64
Roof Material                                      float64
Rooms                                              float64
Bedrooms                                           float64
Basement                                           float64
Basement Finish                                    float64
Central Heating                                    float64
Other Heating                                      float64
Central Air          

,PIN,Property Class,Tax Year,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,...,Location Factor,Garage indicator,Residential share of building,Pure Market Sale,Pure Market Filter,Neigborhood Code (mapping),Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood
0,1011000040000,212,2018,12,11055,10,2.0,5.0,1.0,2.0,...,2.084499,1.0,NaN,0,0,12,105.142760,11.618950,43.634848,1012
1,1011000050000,205,2018,12,6534,10,2.0,0.0,1.0,1.0,...,2.079861,1.0,NaN,0,0,12,80.833162,10.677078,39.522146,1012
2,1011000060000,212,2018,12,6534,10,2.0,3.0,2.0,1.0,...,2.076376,1.0,NaN,0,0,12,80.833162,10.440307,40.951190,1012
3,1011000090000,212,2018,12,13571,10,2.0,2.0,2.0,2.0,...,2.061552,1.0,NaN,0,0,12,116.494635,8.717798,67.416615,1012
4,1011000170000,205,2018,12,9027,10,2.0,0.0,1.0,1.0,...,2.096602,0.0,NaN,0,0,12,95.010526,11.090537,39.799497,1012


In [83]:
# acs = pd.read_csv('data/ACS Data.csv')

col_list = ['PIN', 'Longitude', 'Latitude', 'Property Class', 'Wall Material', 
            'Roof Material', 'Repair Condition', 'Renovation', 
            'Prior Tax Year Market Value Estimate (Land)', 
            'Prior Tax Year Market Value Estimate (Building)', 'Land Square Feet', 
            'Building Square Feet', 'Age']

for col in col_list: 
    print(pipeline.describe(initial_assessments[col]))
    print("-" * 30)
# Important features: 
# Percentages of those in block group: distinct Property Class, 
# distinct wall material, distinct roof material, Repair Condition, Renovation

# Counts: 
# Total units in block group. 

# Medians & Means: 
# Prior Year Market Value Estimate (Land), & Building, Land Square Feet, 
# Building Square Feet, 

pipeline.group_count(initial_assessments, 'Age').sort_values(ascending=False)
# Note that the Assessment data imputes '10' for any houses missing their age. 

count    1.995108e+06
mean     1.433242e+13
std      7.911198e+12
min      1.011000e+12
25%      8.144010e+12
50%      1.405211e+13
75%      1.915217e+13
max      3.332302e+13
Name: PIN, dtype: float64
------------------------------
count    1.991085e+06
mean    -8.779745e+01
std      1.581003e-01
min     -8.826351e+01
25%     -8.788149e+01
50%     -8.776527e+01
75%     -8.767502e+01
max     -8.752481e+01
Name: Longitude, dtype: float64
------------------------------
count    1.991085e+06
mean     4.190541e+01
std      1.649300e-01
min      4.146983e+01
25%      4.179062e+01
50%      4.193405e+01
75%      4.203778e+01
max      4.215398e+01
Name: Latitude, dtype: float64
------------------------------
count    1.995108e+06
mean     2.428938e+02
std      4.303183e+01
min      2.000000e+02
25%      2.030000e+02
50%      2.110000e+02
75%      2.990000e+02
max      2.990000e+02
Name: Property Class, dtype: float64
------------------------------
count    1.393053e+06
mean     1.962359e+00
st

Age
10     60807
62     43847
61     39770
63     39611
64     37510
       ...  
174        1
200        1
203        1
205        1
208        1
Name: Count, Length: 188, dtype: int64

In [84]:
assessments = initial_assessments[col_list]
assessments = gpd.GeoDataFrame(
    assessments, 
    geometry=gpd.points_from_xy(
        assessments['Longitude'], 
        assessments['Latitude']), 
    crs='EPSG:4326')
pipeline.show(assessments)


Shape:
(1995108, 14)

Data Types:
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age                                                   int64
geometry                                           geometry
dtype: object


,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age,geometry
0,1011000040000,-88.139621,42.153953,212,1.0,2.0,2.0,NaN,52510.0,374930.0,11055,1904.0,135,POINT (-88.13962 42.15395)
1,1011000050000,-88.139379,42.153953,205,1.0,1.0,2.0,NaN,31030.0,173970.0,6534,1562.0,114,POINT (-88.13938 42.15395)
2,1011000060000,-88.139196,42.153953,212,2.0,1.0,2.0,NaN,31030.0,353260.0,6534,1677.0,109,POINT (-88.13920 42.15395)
3,1011000090000,-88.138427,42.153945,212,2.0,2.0,2.0,NaN,64460.0,668790.0,13571,4545.0,76,POINT (-88.13843 42.15395)
4,1011000170000,-88.140521,42.153671,205,1.0,1.0,3.0,NaN,42870.0,291310.0,9027,1584.0,123,POINT (-88.14052 42.15367)


In [85]:
acs_g = gpd.read_file('data/geo_census.shp')
pipeline.show(acs_g)


Shape:
(1903, 19)

Data Types:
FIPS           float64
Geographic      object
Census Tra       int64
Block Grou       int64
tot_pop          int64
tot_white        int64
tot_black        int64
hh_size        float64
med_income     float64
occ_units        int64
med_built_     float64
med_rent       float64
oo_hsng_un       int64
GEOID           object
perc_white     float64
perc_non_w     float64
perc_black     float64
perc_owner     float64
geometry      geometry
dtype: object


,FIPS,Geographic,Census Tra,Block Grou,tot_pop,tot_white,tot_black,hh_size,med_income,occ_units,med_built_,med_rent,oo_hsng_un,GEOID,perc_white,perc_non_w,perc_black,perc_owner,geometry
0,1.700000e+11,15000US170310102011,10201,1,1706,479,745,2.30,42778.0,712,1962.0,1097.0,224,170310102011,0.280774,0.719226,0.436694,0.314607,"POLYGON ((-87.68408 42.01515, -87.68387 42.015..."
1,1.700000e+11,15000US170310102012,10201,2,3925,2131,1180,2.69,39535.0,1424,1945.0,1152.0,353,170310102012,0.542930,0.457070,0.300637,0.247893,"POLYGON ((-87.68039 42.01360, -87.68028 42.013..."
2,1.700000e+11,15000US170310102022,10202,2,1591,659,598,2.47,47024.0,620,1946.0,1032.0,117,170310102022,0.414205,0.585795,0.375864,0.188710,"POLYGON ((-87.67599 42.01569, -87.67585 42.015..."
3,1.700000e+11,15000US170310103004,10300,4,1619,1177,224,2.04,39004.0,783,NaN,1072.0,291,170310103004,0.726992,0.273008,0.138357,0.371648,"POLYGON ((-87.67033 42.01683, -87.67033 42.016..."
4,1.700000e+11,15000US170310105011,10501,1,935,469,195,1.81,43516.0,509,NaN,973.0,91,170310105011,0.501604,0.498396,0.208556,0.178782,"POLYGON ((-87.66582 42.01035, -87.66515 42.010..."


In [87]:
assessments = gpd.sjoin(
    left_df=acs_g.loc[:,['GEOID', 'geometry']], 
    right_df=assessments, how='inner', op='intersects')
pipeline.show(assessments)


Shape:
(596793, 16)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age                                                   int64
dtype:

,GEOID,geometry,index_right,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804175,11303150221016,-87.681928,42.01615,299,NaN,NaN,NaN,NaN,6370.0,94860.0,12596,NaN,89
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804187,11303150221001,-87.681928,42.01615,299,NaN,NaN,NaN,NaN,8490.0,126390.0,12596,NaN,89
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804192,11303150221006,-87.681928,42.01615,299,NaN,NaN,NaN,NaN,7630.0,113610.0,12596,NaN,89
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804174,11303150221028,-87.681928,42.01615,299,NaN,NaN,NaN,NaN,630.0,9510.0,12596,NaN,89
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804173,11303150221023,-87.681928,42.01615,299,NaN,NaN,NaN,NaN,5090.0,75830.0,12596,NaN,89


In [89]:
cols_to_dummify = [
    'Property Class', 'Wall Material', 'Roof Material', 'Repair Condition', 
    'Renovation']
assessments = pd.get_dummies(data=assessments, columns=cols_to_dummify, dummy_na=True)
pipeline.show(assessments)


Shape:
(596793, 49)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age                                                   int64
Property Class_200.0                                  uint8
Property Class_201.0                                  uint8
Property Class_202.0                                  uint8
Property Class_203.0                                  uint8
Property Class_204.0                                  uint8
Proper

,GEOID,geometry,index_right,PIN,Longitude,Latitude,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804175,11303150221016,-87.681928,42.01615,6370.0,94860.0,12596,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804187,11303150221001,-87.681928,42.01615,8490.0,126390.0,12596,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804192,11303150221006,-87.681928,42.01615,7630.0,113610.0,12596,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804174,11303150221028,-87.681928,42.01615,630.0,9510.0,12596,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",804173,11303150221023,-87.681928,42.01615,5090.0,75830.0,12596,NaN,...,0,0,1,0,0,0,1,0,0,1


In [90]:
check = assessments == 0
# Drop any columns that 
to_drop = list(check.all()[check.all() == True].index)
assessments.drop(columns=to_drop, inplace=True)
print(f'Dropped the following columns that were all 0:\n{to_drop}')

Dropped the following columns that were all 0:
['Property Class_nan']


In [99]:
# So helpful: https://stackoverflow.com/a/47103408/8527838

cols = list(assessments.columns.values)
cols.pop(cols.index('GEOID'))
assessments = assessments[cols + ['GEOID']]

assessments_1a = assessments.groupby(['GEOID']).agg({
    'PIN': 'count', 
    'Prior Tax Year Market Value Estimate (Land)': ['mean', 'median'], 
    'Prior Tax Year Market Value Estimate (Building)': ['mean', 'median'], 
    'Land Square Feet': ['mean', 'median'],
    'Building Square Feet': ['mean', 'median'], 
    'Age': ['mean', 'median']    
})
assessments_1a

PIN Prior Tax Year Market Value Estimate (Land)           \
             count                                        mean   median   
GEOID                                                                     
170310102011   381                                24320.052493  14510.0   
170310102012   327                                25976.788991   8570.0   
170310102022   288                                32335.243056  13190.0   
170310103004   313                                27782.140575  13360.0   
170310105011   125                                17512.560000   9910.0   
...            ...                                         ...      ...   
170318438002   251                                21348.486056  20310.0   
170318439002    71                                31305.211268  27940.0   
170318439003   413                                 5240.314770   2320.0   
170318439004   152                                26058.552632  15000.0   
170318439005   198                                16334.292929   3610.0   

             Prior Tax Year Market Value Estimate (Building)            \
                                                        mean    median   
GEOID                                                                    
170310102011                                   148836.666667  113610.0   
170310102012                                   153490.152905  112020.0   
170310102022                                   157098.298611  109840.0   
170310103004                                   181857.444089  141300.0   
170310105011                                   145449.440000  108830.0   
...                                                      ...       ...   
170318438002                                   118128.525896  121730.0   
170318439002                                   152449.718310  162210.0   
170318439003                                    40202.227603   37270.0   
170318439004                                    76780.723684   81670.0   
170318439005                                    79534.595960   48740.0   

             Land Square Feet          Building Square Feet          \
                         mean   median                 mean  median   
GEOID                                                                 
170310102011      8630.496063   7662.0          2448.790909  1648.0   
170310102012      8450.584098   6989.0          2083.010000  1805.5   
170310102022     11214.336806   7514.0          2455.865854  2055.5   
170310103004     11298.415335   9000.0          3006.644068  2675.0   
170310105011     17661.600000  22490.0          2349.000000  1822.0   
...                       ...      ...                  ...     ...   
170318438002      3291.223108   3126.0          1658.409836  1160.5   
170318439002      4264.859155   4611.0          4538.304348  4085.5   
170318439003     40131.932203  48225.0          6684.000000  6684.0   
170318439004     19798.881579  18810.0          6038.227273  6243.0   
170318439005     11443.520202  10805.0          6320.550000  6532.5   

                     Age         
                    mean median  
GEOID                            
170310102011   72.958005   88.0  
170310102012   89.287462   92.0  
170310102022   69.000000   55.0  
170310103004   83.610224   95.0  
170310105011   73.448000   69.0  
...                  ...    ...  
170318438002   92.920319  115.0  
170318439002  107.436620  113.0  
170318439003   54.159806   48.0  
170318439004   87.065789   89.0  
170318439005   72.434343   87.0  

[1891 rows x 11 columns]

In [100]:
assessments_1b = (assessments
    .loc[:,'Property Class_200.0':'GEOID']
    .groupby('GEOID').sum())
assessments_1b

,Property Class_200.0,Property Class_201.0,Property Class_202.0,Property Class_203.0,Property Class_204.0,Property Class_205.0,Property Class_206.0,Property Class_207.0,Property Class_208.0,Property Class_209.0,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310102011,0.0,0.0,1.0,15.0,4.0,11.0,3.0,0.0,0.0,0.0,...,0.0,0.0,271.0,0.0,109.0,1.0,271.0,0.0,0.0,381.0
170310102012,0.0,1.0,0.0,9.0,4.0,35.0,2.0,0.0,0.0,0.0,...,0.0,3.0,227.0,0.0,100.0,0.0,227.0,0.0,0.0,327.0
170310102022,0.0,0.0,0.0,2.0,1.0,21.0,19.0,0.0,0.0,0.0,...,4.0,0.0,206.0,2.0,79.0,1.0,206.0,1.0,0.0,287.0
170310103004,0.0,1.0,0.0,1.0,3.0,15.0,25.0,0.0,0.0,0.0,...,4.0,0.0,254.0,0.0,58.0,1.0,254.0,2.0,0.0,311.0
170310105011,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,...,0.0,0.0,114.0,0.0,11.0,0.0,114.0,0.0,0.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170318438002,0.0,2.0,31.0,43.0,3.0,5.0,1.0,0.0,0.0,1.0,...,4.0,4.0,7.0,0.0,237.0,7.0,7.0,0.0,0.0,251.0
170318439002,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,0.0,0.0,25.0,0.0,46.0,0.0,25.0,1.0,0.0,70.0
170318439003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,411.0,0.0,2.0,0.0,411.0,0.0,0.0,413.0


In [101]:
assessments_block_group = assessments_1a.join(other=assessments_1b)
pipeline.show(assessments_block_group)


Shape:
(1891, 48)

Data Types:
(PIN, count)                                                   int64
(Prior Tax Year Market Value Estimate (Land), mean)          float64
(Prior Tax Year Market Value Estimate (Land), median)        float64
(Prior Tax Year Market Value Estimate (Building), mean)      float64
(Prior Tax Year Market Value Estimate (Building), median)    float64
(Land Square Feet, mean)                                     float64
(Land Square Feet, median)                                   float64
(Building Square Feet, mean)                                 float64
(Building Square Feet, median)                               float64
(Age, mean)                                                  float64
(Age, median)                                                float64
Property Class_200.0                                         float64
Property Class_201.0                                         float64
Property Class_202.0                                         float64
Pr

d:\everything\python\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)","(Building Square Feet, median)","(Age, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310102011,381,24320.052493,14510.0,148836.666667,113610.0,8630.496063,7662.0,2448.790909,1648.0,72.958005,...,0.0,0.0,271.0,0.0,109.0,1.0,271.0,0.0,0.0,381.0
170310102012,327,25976.788991,8570.0,153490.152905,112020.0,8450.584098,6989.0,2083.010000,1805.5,89.287462,...,0.0,3.0,227.0,0.0,100.0,0.0,227.0,0.0,0.0,327.0
170310102022,288,32335.243056,13190.0,157098.298611,109840.0,11214.336806,7514.0,2455.865854,2055.5,69.000000,...,4.0,0.0,206.0,2.0,79.0,1.0,206.0,1.0,0.0,287.0
170310103004,313,27782.140575,13360.0,181857.444089,141300.0,11298.415335,9000.0,3006.644068,2675.0,83.610224,...,4.0,0.0,254.0,0.0,58.0,1.0,254.0,2.0,0.0,311.0
170310105011,125,17512.560000,9910.0,145449.440000,108830.0,17661.600000,22490.0,2349.000000,1822.0,73.448000,...,0.0,0.0,114.0,0.0,11.0,0.0,114.0,0.0,0.0,125.0


In [143]:
assessments_block_group.loc[:,'Property Class_200.0':'Renovation_nan'] = (
    assessments_block_group.loc[:,'Property Class_200.0':'Renovation_nan']
        .divide(other=assessments_block_group[('PIN', 'count')], 
                axis=0))
assessments_block_group.head()

,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)","(Building Square Feet, median)","(Age, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310102011,381,24320.052493,14510.0,148836.666667,113610.0,8630.496063,7662.0,2448.790909,1648.0,72.958005,...,0.000000,0.000000,0.711286,0.000000,0.286089,0.002625,0.711286,0.000000,0.0,1.000000
170310102012,327,25976.788991,8570.0,153490.152905,112020.0,8450.584098,6989.0,2083.010000,1805.5,89.287462,...,0.000000,0.009174,0.694190,0.000000,0.305810,0.000000,0.694190,0.000000,0.0,1.000000
170310102022,288,32335.243056,13190.0,157098.298611,109840.0,11214.336806,7514.0,2455.865854,2055.5,69.000000,...,0.013889,0.000000,0.715278,0.006944,0.274306,0.003472,0.715278,0.003472,0.0,0.996528
170310103004,313,27782.140575,13360.0,181857.444089,141300.0,11298.415335,9000.0,3006.644068,2675.0,83.610224,...,0.012780,0.000000,0.811502,0.000000,0.185304,0.003195,0.811502,0.006390,0.0,0.993610
170310105011,125,17512.560000,9910.0,145449.440000,108830.0,17661.600000,22490.0,2349.000000,1822.0,73.448000,...,0.000000,0.000000,0.912000,0.000000,0.088000,0.000000,0.912000,0.000000,0.0,1.000000


In [144]:
final_assessments = (acs_g[['GEOID', 'geometry']].merge(
    right=assessments_block_group, 
    left_on='GEOID', right_index=True))
pipeline.show(final_assessments)


Shape:
(1891, 50)

Data Types:
GEOID                                                          object
geometry                                                     geometry
(PIN, count)                                                    int64
(Prior Tax Year Market Value Estimate (Land), mean)           float64
(Prior Tax Year Market Value Estimate (Land), median)         float64
(Prior Tax Year Market Value Estimate (Building), mean)       float64
(Prior Tax Year Market Value Estimate (Building), median)     float64
(Land Square Feet, mean)                                      float64
(Land Square Feet, median)                                    float64
(Building Square Feet, mean)                                  float64
(Building Square Feet, median)                                float64
(Age, mean)                                                   float64
(Age, median)                                                 float64
Property Class_200.0                                      

,GEOID,geometry,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
0,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015...",381,24320.052493,14510.0,148836.666667,113610.0,8630.496063,7662.0,2448.790909,...,0.000000,0.000000,0.711286,0.000000,0.286089,0.002625,0.711286,0.000000,0.0,1.000000
1,170310102012,"POLYGON ((-87.68039 42.01360, -87.68028 42.013...",327,25976.788991,8570.0,153490.152905,112020.0,8450.584098,6989.0,2083.010000,...,0.000000,0.009174,0.694190,0.000000,0.305810,0.000000,0.694190,0.000000,0.0,1.000000
2,170310102022,"POLYGON ((-87.67599 42.01569, -87.67585 42.015...",288,32335.243056,13190.0,157098.298611,109840.0,11214.336806,7514.0,2455.865854,...,0.013889,0.000000,0.715278,0.006944,0.274306,0.003472,0.715278,0.003472,0.0,0.996528
3,170310103004,"POLYGON ((-87.67033 42.01683, -87.67033 42.016...",313,27782.140575,13360.0,181857.444089,141300.0,11298.415335,9000.0,3006.644068,...,0.012780,0.000000,0.811502,0.000000,0.185304,0.003195,0.811502,0.006390,0.0,0.993610
4,170310105011,"POLYGON ((-87.66582 42.01035, -87.66515 42.010...",125,17512.560000,9910.0,145449.440000,108830.0,17661.600000,22490.0,2349.000000,...,0.000000,0.000000,0.912000,0.000000,0.088000,0.000000,0.912000,0.000000,0.0,1.000000
